In [1]:
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
from sklearn.datasets import fetch_20newsgroups
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

c:\users\zjyju\appdata\local\programs\python\python38\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


docs = fetch_20newsgroups(subset = 'all',  remove = ('headers', 'footers', 'quotes'))['data']

https://github.com/sethns/Latent-Dirichlet-Allocation-LDA-/blob/main/Topic%20Modeling%20_%20Extracting%20Topics_%20Using%20Genism.ipynb

In [2]:
D1 = 'I want to watch a movie this weekend.'
D2 =  'I went shopping yesterday. New Zealand won the World Test Championship by beating India by eight wickets at Southampton.'
D3 =  'I don’t watch cricket. Netflix and Amazon Prime have very good movies to watch.'
D4 =  'Movies are a nice way to chill however, this time I would like to paint and read some good books. It’s been long!'
D5 =  'This blueberry milkshake is so good! Try reading Dr. Joe Dispenza’s books. His work is such a game-changer! His books helped to learn so much about how our thoughts impact our biology and how we can all rewire our brains.'

corpus = [D1, D2, D3, D4, D5]

In [3]:
###
#   Performs lemmization of input documents.
#   Args:
#     - docs: list of strings with input documents
#     - allowed_postags: list of accepted Part of Speech (POS) types
#   Output:
#     - list of strings with lemmatized input
###
def lemmatize(docs, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable = ["parser", "ner"])
    lemmatized_docs = []
    for doc in docs:
        doc = nlp(doc)
        tokens = []
    for token in doc:
        if token.pos_ in allowed_postags:
            tokens.append(token.lemma_)
        lemmatized_docs.append(" ".join(tokens))
    return (lemmatized_docs)

In [4]:
def tokenize(docs):
#   '''
#   Performs tokenization of input documents.
#   Args:
#     - docs: list of strings with input documents
#   Output:
#     - list of strings with tokenized input
#   '''
    tokenized_docs = []
    for doc in docs:
        tokens = gensim.utils.simple_preprocess(doc, deacc=True)
        tokenized_docs.append(tokens)
    return (tokenized_docs)

In [5]:
# Pre-process input: lemmatization and tokenization
lemmatized_docs = lemmatize(corpus)
tokenized_docs = tokenize(lemmatized_docs)

c:\users\zjyju\appdata\local\programs\python\python38\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [6]:
# Mapping from word IDs to words
id2word = corpora.Dictionary(tokenized_docs)

In [7]:
# Prepare Document-Term Matrix
corpus1 = []
for doc in tokenized_docs:
    corpus1.append(id2word.doc2bow(doc))

In [8]:
# Fit LDA model: See [1] for more details
topic_model = gensim.models.ldamodel.LdaModel(
    corpus = corpus1,      # Document-Term Matrix
    id2word = id2word,    # Map word IDs to words
    num_topics = 10,      # Number of latent topics to extract
    random_state = 0,
    passes = 1,         # N° of passes through the corpus during training
    eval_every=None
    )

In [9]:
# Visualize with pyLDAvis: See [2] for more details
pyLDAvis.enable_notebook()
visualization = pyLDAvis.gensim_models.prepare(
    topic_model, 
    corpus1,
    id2word, 
    mds = "mmds", 
    R = 30)

c:\users\zjyju\appdata\local\programs\python\python38\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [10]:
visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.012274  0.010653       1        1  40.027320
7     -0.022849  0.043389       2        1  29.245736
4     -0.089022 -0.000123       3        1  14.685066
3     -0.020411  0.017403       4        1   8.610344
2     -0.014976 -0.095206       5        1   3.085092
6     -0.011616  0.024730       6        1   1.183893
8     -0.055621 -0.055168       7        1   0.790757
1      0.067514  0.017351       8        1   0.790597
5      0.067354  0.018480       9        1   0.790597
9      0.067352  0.018490      10        1   0.790597, topic_info=         Term       Freq      Total Category  logprob  loglift
2          so  60.000000  60.000000  Default  19.0000  19.0000
0   blueberry  45.000000  45.000000  Default  18.0000  18.0000
1   milkshake  44.000000  44.000000  Default  17.0000  17.0000
3        good  44.000000  44.000000  Default  16.0000  16.0000
4         try  40.000000  40.000000  Default  15.0000  15.0000
..        ...        ...        ...      ...      ...      ...
3        good   0.214382  44.273198  Topic10  -2.9440  -0.4902
1   milkshake   0.214542  44.445815  Topic10  -2.9433  -0.4934
0   blueberry   0.214594  45.578520  Topic10  -2.9430  -0.5183
6        book   0.214399  53.825217  Topic10  -2.9439  -0.6855
2          so   0.214472  60.450850  Topic10  -2.9436  -0.8013

[209 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
16        1  0.839025        all
15        1  0.690608    biology
15        4  0.115101    biology
0         1  0.372983  blueberry
0         2  0.241342  blueberry
...     ...       ...        ...
4         7  0.024708        try
7         1  0.312860       work
7         2  0.312860       work
7         3  0.255976       work
7         4  0.085325       work

[72 rows x 3 columns], R=19, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 5, 4, 3, 7, 9, 2, 6, 10])

In [140]:

df=pd.DataFrame(topic_model.get_document_topics(corpus1))
df

,0,1,2,3,4,5,6,7,8,9
0,"(0, 0.1)","(1, 0.1)","(2, 0.1)","(3, 0.1)","(4, 0.1)","(5, 0.1)","(6, 0.1)","(7, 0.1)","(8, 0.1)","(9, 0.1)"
1,"(0, 0.05001497)","(1, 0.05000006)","(2, 0.54990304)","(3, 0.050016556)","(4, 0.050017882)","(5, 0.05000006)","(6, 0.050017655)","(7, 0.050012976)","(8, 0.050016697)","(9, 0.05000006)"
2,"(0, 0.033341113)","(1, 0.033333372)","(2, 0.6999437)","(3, 0.03334223)","(4, 0.033342827)","(5, 0.033333372)","(6, 0.03334736)","(7, 0.033340316)","(8, 0.03334232)","(9, 0.033333372)"
3,"(0, 0.033341106)","(1, 0.033333372)","(2, 0.69994634)","(3, 0.033342227)","(4, 0.033343922)","(5, 0.033333372)","(6, 0.03334285)","(7, 0.03334031)","(8, 0.033343136)","(9, 0.033333372)"
4,"(0, 0.025006177)","(1, 0.025000028)","(2, 0.77496064)","(3, 0.025006786)","(4, 0.025006618)","(5, 0.025000028)","(6, 0.02500734)","(7, 0.02500608)","(8, 0.025006266)","(9, 0.025000028)"
5,"(0, 0.02000459)","(1, 0.02000002)","(2, 0.81997085)","(3, 0.020005131)","(4, 0.02000516)","(5, 0.02000002)","(6, 0.020004904)","(7, 0.020004481)","(8, 0.020004839)","(9, 0.02000002)"
6,"(0, 0.020004585)","(1, 0.02000002)","(2, 0.81997085)","(3, 0.020005219)","(4, 0.020005116)","(5, 0.02000002)","(6, 0.020004906)","(7, 0.02000448)","(8, 0.02000484)","(9, 0.02000002)"
7,"(0, 0.016671361)","(1, 0.01666669)","(2, 0.016672688)","(3, 0.849969)","(4, 0.016672188)","(5, 0.01666669)","(6, 0.016671536)","(7, 0.016671235)","(8, 0.016671894)","(9, 0.01666669)"
8,"(0, 0.01428957)","(1, 0.014285733)","(2, 0.014289672)","(3, 0.014290138)","(4, 0.87140334)","(5, 0.014285733)","(6, 0.014289616)","(7, 0.014289509)","(8, 0.01429094)","(9, 0.014285733)"
9,"(0, 0.014289681)","(1, 0.014285733)","(2, 0.014289702)","(3, 0.8714033)","(4, 0.014290593)","(5, 0.014285733)","(6, 0.014289717)","(7, 0.014289593)","(8, 0.014290246)","(9, 0.014285733)"


In [169]:
#pd.DataFrame(df.loc[2].tolist()).set_index(0).transpose()
df

,0,1,2,3,4,5,6,7,8,9
0,"(0, 0.1)","(1, 0.1)","(2, 0.1)","(3, 0.1)","(4, 0.1)","(5, 0.1)","(6, 0.1)","(7, 0.1)","(8, 0.1)","(9, 0.1)"
1,"(0, 0.05001497)","(1, 0.05000006)","(2, 0.54990304)","(3, 0.050016556)","(4, 0.050017882)","(5, 0.05000006)","(6, 0.050017655)","(7, 0.050012976)","(8, 0.050016697)","(9, 0.05000006)"
2,"(0, 0.033341113)","(1, 0.033333372)","(2, 0.6999437)","(3, 0.03334223)","(4, 0.033342827)","(5, 0.033333372)","(6, 0.03334736)","(7, 0.033340316)","(8, 0.03334232)","(9, 0.033333372)"
3,"(0, 0.033341106)","(1, 0.033333372)","(2, 0.69994634)","(3, 0.033342227)","(4, 0.033343922)","(5, 0.033333372)","(6, 0.03334285)","(7, 0.03334031)","(8, 0.033343136)","(9, 0.033333372)"
4,"(0, 0.025006177)","(1, 0.025000028)","(2, 0.77496064)","(3, 0.025006786)","(4, 0.025006618)","(5, 0.025000028)","(6, 0.02500734)","(7, 0.02500608)","(8, 0.025006266)","(9, 0.025000028)"
5,"(0, 0.02000459)","(1, 0.02000002)","(2, 0.81997085)","(3, 0.020005131)","(4, 0.02000516)","(5, 0.02000002)","(6, 0.020004904)","(7, 0.020004481)","(8, 0.020004839)","(9, 0.02000002)"
6,"(0, 0.020004585)","(1, 0.02000002)","(2, 0.81997085)","(3, 0.020005219)","(4, 0.020005116)","(5, 0.02000002)","(6, 0.020004906)","(7, 0.02000448)","(8, 0.02000484)","(9, 0.02000002)"
7,"(0, 0.016671361)","(1, 0.01666669)","(2, 0.016672688)","(3, 0.849969)","(4, 0.016672188)","(5, 0.01666669)","(6, 0.016671536)","(7, 0.016671235)","(8, 0.016671894)","(9, 0.01666669)"
8,"(0, 0.01428957)","(1, 0.014285733)","(2, 0.014289672)","(3, 0.014290138)","(4, 0.87140334)","(5, 0.014285733)","(6, 0.014289616)","(7, 0.014289509)","(8, 0.01429094)","(9, 0.014285733)"
9,"(0, 0.014289681)","(1, 0.014285733)","(2, 0.014289702)","(3, 0.8714033)","(4, 0.014290593)","(5, 0.014285733)","(6, 0.014289717)","(7, 0.014289593)","(8, 0.014290246)","(9, 0.014285733)"


In [168]:
x=pd.melt(df.reset_index(),id_vars="index").dropna()
x['topic'],x['score']=x.value.str 
x.pivot(index='index',columns='topic')['score']

C:\Users\zjyju\AppData\Local\Temp\ipykernel_580196\3100273805.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  x['topic'],x['score']=x.value.str #.apply(lambda r: pd.DataFrame({'c_id':r['index'], 'topic':r.value[0], 'score':r.value[1]}), axis=1)


topic,0,1,2,3,4,5,6,7,8,9
index,,,,,,,,,,
0,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0.050015,0.050000,0.549903,0.050017,0.050018,0.050000,0.050018,0.050013,0.050017,0.050000
2,0.033341,0.033333,0.699944,0.033342,0.033343,0.033333,0.033347,0.033340,0.033342,0.033333
3,0.033341,0.033333,0.699946,0.033342,0.033344,0.033333,0.033343,0.033340,0.033343,0.033333
4,0.025006,0.025000,0.774961,0.025007,0.025007,0.025000,0.025007,0.025006,0.025006,0.025000
5,0.020005,0.020000,0.819971,0.020005,0.020005,0.020000,0.020005,0.020004,0.020005,0.020000
6,0.020005,0.020000,0.819971,0.020005,0.020005,0.020000,0.020005,0.020004,0.020005,0.020000
7,0.016671,0.016667,0.016673,0.849969,0.016672,0.016667,0.016672,0.016671,0.016672,0.016667
8,0.014290,0.014286,0.014290,0.014290,0.871403,0.014286,0.014290,0.014290,0.014291,0.014286
